<a href="https://colab.research.google.com/github/manthanvaghela/AI-Driven-Cybersecurity-Threat-Prediction-Platform/blob/main/SQL_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sqlite3
import pandas as pd

connection = sqlite3.connect(":memory:")
cur = connection.cursor()

Create a table employees with columns: id, name, department, salary, join_date. Insert 5 sample rows.


In [4]:
cur.execute("""
CREATE TABLE employees (
    id INTEGER,
    name TEXT,
    department TEXT,
    salary INTEGER,
    join_date TEXT
)
""")

records = [
    (1, "Asha", "HR", 50000, "2022-01-10"),
    (2, "Rahul", "IT", 70000, "2021-11-05"),
    (3, "Priya", "Finance", 65000, "2023-03-15"),
    (4, "Kiran", "IT", 72000, "2020-07-20"),
    (5, "Meena", "HR", 52000, "2022-06-30")
]

cur.executemany("INSERT INTO employees VALUES (?, ?, ?, ?, ?)", records)


Prompt: Show all employees from the IT department earning more than ₹70,000.

In [5]:
sql = "SELECT * FROM employees WHERE department = 'IT' AND salary > 70000"
it_staff = pd.read_sql(sql, connection)
print(it_staff)

   id   name department  salary   join_date
0   4  Kiran         IT   72000  2020-07-20


Prompt:Show average salary per department using group by

In [6]:
sql_cmd = "SELECT department, AVG(salary) AS avg_salary FROM employees GROUP BY department"
dept_avg_salary = pd.read_sql(sql_cmd, connection)
print(dept_avg_salary)


  department  avg_salary
0    Finance     65000.0
1         HR     51000.0
2         IT     71000.0


Prompt: List employees by highest salary using order by


In [ ]:
sql_stmt = "SELECT * FROM employees ORDER BY salary DESC"
sorted_employees = pd.read_sql(sql_stmt, connection)
print(sorted_employees)


,id,name,department,salary,join_date
0,4,Kiran,IT,72000,2020-07-20
1,2,Rahul,IT,70000,2021-11-05
2,3,Priya,Finance,65000,2023-03-15
3,5,Meena,HR,52000,2022-06-30
4,1,Asha,HR,50000,2022-01-10


Prompt:Show employees who joined after Jan 1, 2022.

In [7]:
recent_employees = pd.read_sql(
    "SELECT * FROM employees WHERE join_date > '2022-01-01'",
    connection
)
print(recent_employees)


   id   name department  salary   join_date
0   1   Asha         HR   50000  2022-01-10
1   3  Priya    Finance   65000  2023-03-15
2   5  Meena         HR   52000  2022-06-30


Prompt: Create a second table projects and join with employees to show who is working on what.


In [8]:
cur.execute("""
CREATE TABLE projects (
    emp_id INTEGER,
    project_name TEXT
)
""")

project_data = [
    (1, "Onboarding"),
    (2, "AI Chatbot"),
    (3, "Budget Tracker"),
    (4, "Cyber Threat Agent")
]

cur.executemany("INSERT INTO projects VALUES (?, ?)", project_data)

joined_data = pd.read_sql(
    """
    SELECT e.name, e.department, p.project_name
    FROM employees AS e
    INNER JOIN projects AS p ON e.id = p.emp_id
    """,
    connection
)

print(joined_data)


    name department        project_name
0   Asha         HR          Onboarding
1  Rahul         IT          AI Chatbot
2  Priya    Finance      Budget Tracker
3  Kiran         IT  Cyber Threat Agent


Prompt: show inner join right join left join self join with example

### Inner Join

An INNER JOIN returns only the rows where there is a match in both tables.

In [9]:
sql_join = """
SELECT e.name, e.department, p.project_name
FROM employees AS e
JOIN projects AS p ON e.id = p.emp_id
"""
joined_df = pd.read_sql(sql_join, connection)
print(joined_df)

    name department        project_name
0   Asha         HR          Onboarding
1  Rahul         IT          AI Chatbot
2  Priya    Finance      Budget Tracker
3  Kiran         IT  Cyber Threat Agent


### Left Join

A LEFT JOIN returns all rows from the left table, and the matched rows from the right table. If there is no match in the right table, the result is NULL on the right side.

In [10]:
sql_left = """
SELECT e.name, e.department, p.project_name
FROM employees AS e
LEFT JOIN projects AS p ON e.id = p.emp_id
"""
left_join_result = pd.read_sql(sql_left, connection)
print(left_join_result)


    name department        project_name
0   Asha         HR          Onboarding
1  Rahul         IT          AI Chatbot
2  Priya    Finance      Budget Tracker
3  Kiran         IT  Cyber Threat Agent
4  Meena         HR                None


### Right Join

A RIGHT JOIN returns all rows from the right table, and the matched rows from the left table. If there is no match in the left table, the result is NULL on the left side.

*Note: SQLite does not directly support RIGHT JOIN. We can achieve a similar result by switching the table order and using a LEFT JOIN.*

In [11]:
sql_right = """
SELECT e.name, e.department, p.project_name
FROM projects AS p
LEFT JOIN employees AS e ON p.emp_id = e.id
"""
right_join_result = pd.read_sql(sql_right, connection)
print(right_join_result)


    name department        project_name
0   Asha         HR          Onboarding
1  Rahul         IT          AI Chatbot
2  Priya    Finance      Budget Tracker
3  Kiran         IT  Cyber Threat Agent


### Self Join

A SELF JOIN is a regular join used to join a table to itself. It is used to compare rows within the same table.

In [12]:
sql_self = """
SELECT a.name AS emp1, b.name AS emp2, a.department
FROM employees AS a
JOIN employees AS b ON a.department = b.department
WHERE a.id != b.id
"""
self_join_result = pd.read_sql(sql_self, connection)
print(self_join_result)


    emp1   emp2 department
0   Asha  Meena         HR
1  Rahul  Kiran         IT
2  Kiran  Rahul         IT
3  Meena   Asha         HR


Prompt: Show each employee’s salary and compare it with the previous and next employee.


In [13]:
cur.execute("""
CREATE TABLE salaries (
    id INTEGER,
    name TEXT,
    salary INTEGER
)
""")

salary_data = [
    (1, "Asha", 50000),
    (2, "Rahul", 70000),
    (3, "Priya", 65000),
    (4, "Kiran", 72000),
    (5, "Meena", 52000)
]

cur.executemany("INSERT INTO salaries VALUES (?, ?, ?)", salary_data)

window_query = """
SELECT
    id,
    name,
    salary,
    LAG(salary) OVER (ORDER BY id) AS previous_salary,
    LEAD(salary) OVER (ORDER BY id) AS following_salary
FROM salaries
"""

salary_window_df = pd.read_sql(window_query, connection)
print(salary_window_df)


   id   name  salary  previous_salary  following_salary
0   1   Asha   50000              NaN           70000.0
1   2  Rahul   70000          50000.0           65000.0
2   3  Priya   65000          70000.0           72000.0
3   4  Kiran   72000          65000.0           52000.0
4   5  Meena   52000          72000.0               NaN


Prompt: Show names that appear more than once.(Find the duplicate values)


In [14]:
sql_duplicates = "SELECT name, COUNT(*) AS occurrences FROM employees GROUP BY name HAVING COUNT(*) > 1"
dup_names = pd.read_sql(sql_duplicates, connection)
print(dup_names)


Empty DataFrame
Columns: [name, occurrences]
Index: []


Prompt: Convert department-wise monthly expenses into columns.


In [15]:
cur.execute("""
CREATE TABLE monthly_expenses (
    month TEXT,
    department TEXT,
    expense INTEGER
)
""")

expenses_data = [
    ("Jan", "HR", 10000),
    ("Jan", "IT", 25000),
    ("Jan", "Finance", 15000),
    ("Feb", "HR", 12000),
    ("Feb", "IT", 27000),
    ("Feb", "Finance", 16000),
    ("Mar", "HR", 11000),
    ("Mar", "IT", 26000),
    ("Mar", "Finance", 15500)
]

cur.executemany("INSERT INTO monthly_expenses VALUES (?, ?, ?)", expenses_data)

sql_pivot = """
SELECT
    month,
    SUM(CASE WHEN department = 'HR' THEN expense ELSE 0 END) AS hr_total,
    SUM(CASE WHEN department = 'IT' THEN expense ELSE 0 END) AS it_total,
    SUM(CASE WHEN department = 'Finance' THEN expense ELSE 0 END) AS finance_total
FROM monthly_expenses
GROUP BY month
ORDER BY month
"""

pivot_result = pd.read_sql(sql_pivot, connection)
print(pivot_result)


  month  hr_total  it_total  finance_total
0   Feb     12000     27000          16000
1   Jan     10000     25000          15000
2   Mar     11000     26000          15500


Prompt:Unpivot it

In [16]:
unpivoted_df = pivot_result.melt(id_vars=["month"], var_name="department", value_name="expense")
print(unpivoted_df)


  month     department  expense
0   Feb       hr_total    12000
1   Jan       hr_total    10000
2   Mar       hr_total    11000
3   Feb       it_total    27000
4   Jan       it_total    25000
5   Mar       it_total    26000
6   Feb  finance_total    16000
7   Jan  finance_total    15000
8   Mar  finance_total    15500


Prompt: Combine first and last names into full name
by doing concatenation

In [17]:
cur.execute("""
CREATE TABLE people (
    first_name TEXT,
    last_name TEXT
)
""")

names_data = [
    ("Vivek", "Sharma"),
    ("Asha", "Rao"),
    ("Rahul", "Mehta")
]

cur.executemany("INSERT INTO people VALUES (?, ?)", names_data)

sql_fullname = "SELECT first_name || ' ' || last_name AS full_name FROM people"
full_name_df = pd.read_sql(sql_fullname, connection)
print(full_name_df)


      full_name
0  Vivek Sharma
1      Asha Rao
2   Rahul Mehta
